In [2]:
import os
import pandas as pd
import qiime2 as q2
from qiime2 import Visualization

%matplotlib inline

In [3]:
data = 'Data'

In [4]:
data_classified = 'Data/classified'

In [5]:
data_raw = 'Data/raw'

In [6]:
data_alpha = 'Data/diversity'

In [23]:
! qiime diversity alpha-rarefaction \
    --i-table $data_classified/table-filtered.qza \
    --p-max-depth 10000 \
    --p-iterations 10 \
    --m-metadata-file $data_raw/20250913_metadata_ITS.tsv \
    --o-visualization $data_alpha/alpha-rarefaction.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: Data/diversity/alpha-rarefaction.qzv


In [ ]:
###################################################################


# hier ist der Vorschlag von Annina

In [24]:
Visualization.load(f"{data_alpha}/alpha-rarefaction.qzv")

<visualization: Visualization uuid: 1ec9a82b-7309-4564-9f43-c6e9e6d7ec18>

In [ ]:
#Bis hier ist alles gut!!!

#Versuch die Metadaten erst nachher zu mergen

In [20]:
import os
os.makedirs("Data/diversity/kmerizer_results", exist_ok=True)

In [24]:
! qiime kmerizer seqs-to-kmers \
  --i-sequences Data/classified/rep-seqs-filtered.qza \
  --i-table Data/diversity/core-metrics-results/rarefied_table.qza \
  --p-kmer-size 100 \
  --p-no-tfidf True \
  --o-kmer-table Data/diversity/kmerizer_results/kmer-table.qza 

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [ ]:
! qiime diversity alpha \
  --i-table $data_diversity/kmerizer_results/kmer_table.qza \
  --p-metric pielou_e \
  --o-alpha-diversity $data_diversity/core-metrics-results/pielou_e.qza

In [ ]:
! qiime diversity alpha \
  --i-table $data_diversity/kmerizer_results/kmer_table.qza \
  --p-metric observed_features \
  --o-alpha-diversity $data_diversity/kmerizer_results/observed_features.qza

In [ ]:
! qiime diversity alpha \
  --i-table $data_diversity/kmerizer_results/kmer_table.qza \
  --p-metric shannon \
  --o-alpha-diversity $data_diversity/kmerizer_results/shannon.qza

In [25]:

#######################################################################################




#Ab hier ist wie in Übung





In [8]:
! qiime diversity core-metrics \
  --i-table $data_classified/table-filtered.qza \
  --m-metadata-file $data_raw/20250913_metadata_ITS.tsv \
  --p-sampling-depth 3000 \
  --output-dir $data_alpha/core-metrics-results

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Usage: qiime diversity core-metrics [OPTIONS]

  Applies a collection of diversity metrics (non-phylogenetic) to a feature
  table.

Inputs:
  --i-table ARTIFACT FeatureTable[Frequency]
                          The feature table containing the samples over which
                          diversity metrics should be computed.     [required]
Parameters:
  --p-sampling-depth INTEGER
    Range(1, None)        The total frequency that each sample should be
                          rarefied to prior to computing diversity metrics.
                                                                    [required]
  --m-metadata-file METADATA...
    (multiple arguments   Th

In [7]:
! qiime kmerizer core-metrics \
  --i-table $data_classified/table-filtered.qza \
  --i-sequences $data_classified/rep-seqs-filtered.qza \
  --p-sampling-depth 3000 \
  --p-kmer-size 8 \
  --m-metadata-file Data/raw/20250913_metadata_ITS.tsv \
  --output-dir $data_alpha/kmerizer-results

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [38]:
ls $data_alpha/kmerizer-results

ls: cannot access 'Data/diversity/kmerizer-results': No such file or directory
